In [ ]:
%matplotlib notebook

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import pandas as pd
import yaml
from dateutil.parser import parse

from skynet.utils.enums import Power
from skynet.prediction import climate_model, predict

from skynet.sample import sample

from skynet.utils.storage import get_storage
from skynet.utils import thermo
from skynet.utils.script_utils import get_connections

from utils.climate_model_predict import create_climate_model_predictions_for_history_features, get_history_feature_for_timestamps
from ipywidgets import interact, fixed
import ipywidgets as widgets

In [ ]:
plt.rcParams["figure.figsize"] = 16, 10

In [ ]:
cnf = yaml.load(open('../config.yml'))
connections = get_connections(config=cnf, mysql='viewer', cassandra='viewer')
model_store = get_storage(storage="file", **cnf["model_store"], directory="../data/models")
predictor = predict.Predictor(model_store.load(climate_model.ClimateModel.get_storage_key()))

In [ ]:
def get_history_feature_for_a_single_timestamp(history_features, timestamps, timestamp):
    return history_features[timestamps.index(timestamp)]

In [ ]:
STATES = [
    {"mode": mode, "temperature": temperature_set, "power": Power.ON}
    for mode in ["heat", "cool", "fan", "auto", "dry", "off"]
    for temperature_set in range(16, 33)
]

FEATURE_COLUMNS = [
    "humidex",
    "humidity",
    "temperature_out",
    "temperature",
    "temperature_out_mean_day",
    "temperature_delta",
]

COLORMAP = {
    "off": "grey",
    "heat": "red",
    "cool": "green",
    "dry": "purple",
    "fan": "pink",
    "auto": "black"
}


In [ ]:
device_id = "1dfeea44-790d-4061-b057-b09f43a078f4"

# current_timestamp = parse(datetime.utcnow().strftime("%Y-%m-%d %H:%M"))
current_timestamp = parse('2019-03-15 01:30')
continuous_update = False

In [ ]:
timestamps = [current_timestamp - timedelta(seconds=60*x) for x in range(-100, 101, 5)]


history_features = await get_history_feature_for_timestamps(device_id, timestamps, connections)
current_history_feature = get_history_feature_for_a_single_timestamp(history_features, timestamps, current_timestamp)

In [ ]:
timestamp_slider = widgets.SelectionSlider(description='Timestamp', value=current_timestamp, options=timestamps, layout= widgets.Layout(width="900px"), continuous_update=continuous_update)

temperature_slider=widgets.FloatSlider(value=current_history_feature['temperature'], min=-10.0, max=40.0, step=0.1, description="temp", layout= widgets.Layout(width="900px"), continuous_update=continuous_update)
humidity_slider=widgets.IntSlider(value=current_history_feature['humidity'], min=10.0, max=90.0, step=4, description="humidity", layout= widgets.Layout(width="900px"), continuous_update=continuous_update)

temperature_out_slider=widgets.FloatSlider(value=current_history_feature['temperature_out'], min=-10.0, max=40.0, step=1, description="temp out", layout= widgets.Layout(width="900px"), continuous_update=continuous_update)

In [ ]:
def update_history_feature(history_feature, temperature, humidity, temperature_out):
    history_feature = {**history_feature}
    
    history_feature.update({
        "temperature_out": temperature_out,
        "temperature": temperature,
        "humidity": humidity,
        "humidex": thermo.humidex(temperature, humidity)
    })

    return {**history_feature}


    
x = [state.get('temperature', None) for state in STATES if state["mode"] == "off"]

modes = ["off", "heat", "fan", "dry", "cool", "auto"]

default_prediction = create_climate_model_predictions_for_history_features([current_history_feature], predictor, [current_timestamp])

In [ ]:
fig = plt.figure()
ax_temperature = fig.add_subplot(1, 3, 1)
ax_humidity = fig.add_subplot(1, 3, 2)
ax_humidex = fig.add_subplot(1, 3, 3)

ax_temperature.set_ylabel("Temperautre")
ax_humidity.set_ylabel("Humidity")
ax_humidex.set_ylabel("Humidex")

ax_temperature.set_xlabel("Set Temperature")
ax_humidity.set_xlabel("Set Temperature")
ax_humidex.set_xlabel("Set Temperature")

ax_temperature.set_ylim(default_prediction["temperature"].min() - 5, default_prediction["temperature"].max() + 5)
ax_humidity.set_ylim(default_prediction["humidity"].min() - 10, default_prediction["humidity"].max() + 10)
ax_humidex.set_ylim(default_prediction["humidex"].min() - 5, default_prediction["humidex"].max() + 5)

for mode in modes:
    temperature_line, = ax_temperature.plot(x, default_prediction.loc[default_prediction["mode"] == mode]['temperature'], color=COLORMAP[mode], marker="o", linestyle=":", label=mode)
    humidity_line, = ax_humidity.plot(x, default_prediction.loc[default_prediction["mode"] == mode]['humidity'], color=COLORMAP[mode], marker="o", linestyle=":", label=mode)
    humidex_line, = ax_humidex.plot(x, default_prediction.loc[default_prediction["mode"] == mode]['humidex'], color=COLORMAP[mode], marker="o", linestyle=":", label=mode)


temperature_line = ax_temperature.plot(x, [current_history_feature["temperature"]] * len(x), color='k', linestyle="--", label="current value")
humidity_line = ax_humidity.plot(x, [current_history_feature["humidity"]] * len(x), color='k', linestyle="--", label="current value")
humidex_line = ax_humidex.plot(x, [current_history_feature["humidex"]] * len(x), color='k', linestyle="--", label="current value")

    
ax_temperature.legend(modes)

def update_climate_predictions(history_features, predictor, timestamps, timestamp, temperature, humidity, temperature_out):
    history_feature = get_history_feature_for_a_single_timestamp(history_features, timestamps, timestamp_slider.value)
    
    print(f"Original temp: {history_feature['temperature']}, humidity: {history_feature['humidity']}, humidex: {history_feature['humidex']}")
    
    updated_history_feature = update_history_feature(history_feature, temperature, humidity, temperature_out)
    
    prediction = create_climate_model_predictions_for_history_features([updated_history_feature], predictor, [current_timestamp])
    
    print("Current Input Features: " + ", ".join([f"{feature}: {updated_history_feature.get(feature, '---')}" for feature in FEATURE_COLUMNS]))
    
    ax_temperature.lines[-1].set_data(x, [updated_history_feature["temperature"]] * len(x))
    ax_humidity.lines[-1].set_data(x, [updated_history_feature["humidity"]] * len(x))
    ax_humidex.lines[-1].set_data(x, [updated_history_feature["humidex"]] * len(x))

    for index, mode in enumerate(modes):
        ax_temperature.lines[index].set_data(x, prediction.loc[prediction["mode"] == mode]["temperature"])
        ax_humidity.lines[index].set_data(x, prediction.loc[prediction["mode"] == mode]["humidity"])
        ax_humidex.lines[index].set_data(x, prediction.loc[prediction["mode"] == mode]["humidex"])

    fig.canvas.draw()

def update_default_value_for_new_timestamp(*args, history_features=history_features, timestamps=timestamps):
                     
    history_feature = get_history_feature_for_a_single_timestamp(history_features, timestamps, timestamp_slider.value)

    temperature_slider.value = history_feature.get("temperature", 25) if history_feature else 25
    humidity_slider.value = history_feature.get("humidity", 50) if history_feature else 50
    temperature_out_slider.value = history_feature.get("temperature_out", 10) if history_feature else 10
    

timestamp_slider.observe(update_default_value_for_new_timestamp, "value")

interact(
    update_climate_predictions, 
    predictor=fixed(predictor),
    current_timestamp = fixed(current_timestamp),
    timestamps=fixed(timestamps),
    history_features=fixed(history_features),
    timestamp=timestamp_slider,
    temperature=temperature_slider,
    humidity=humidity_slider,
    temperature_out=temperature_out_slider)